In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="iBtjlD2F0Clrr7okqCCf")
project = rf.workspace("sunflower-detection").project("final-1yr0d")
version = project.version(3)
dataset = version.download("yolov8")
                

In [ ]:
%pip install ultralytics supervision roboflow
import ultralytics
ultralytics.checks()

In [ ]:
!sed -i '$d' {dataset.location}/data.yaml   # Delete the last line
!sed -i '$d' {dataset.location}/data.yaml   # Delete the second-to-last line
!sed -i '$d' {dataset.location}/data.yaml   # Delete the third-to-last line

!echo 'test: ../test/images' >> {dataset.location}/data.yaml
!echo 'train: ../train/images' >> {dataset.location}/data.yaml
!echo 'val: ../valid/images' >> {dataset.location}/data.yaml

In [ ]:
!yolo task=detect mode=train model=/kaggle/input/yolov8s/yolov8s.pt data=/kaggle/working/Final-3/data.yaml epochs=50 imgsz=640 plots=True

In [ ]:
from IPython.display import Image as IPyImage

IPyImage(filename=f'/kaggle/working/runs/detect/train/confusion_matrix.png', width=600)

In [ ]:
!yolo task=detect mode=predict model=/kaggle/working/runs/detect/train/weights/best.pt conf=0.25 source=/kaggle/working/Final-3/test/images save=True

In [ ]:
import glob
import os
from IPython.display import Image as IPyImage, display

latest_folder = max(glob.glob('/kaggle/working/runs/detect/predict*/'), key=os.path.getmtime)
for img in glob.glob(f'{latest_folder}/*.jpg')[:3]:
	display(IPyImage(filename=img, width=600))
	print("\n")

In [ ]:
display(IPyImage(filename='/kaggle/working/runs/detect/train/results.png', width=600))

In [ ]:
import pandas as pd
from tabulate import tabulate


results_csv_path = '/kaggle/working/runs/detect/train/results.csv'
results_df = pd.read_csv(results_csv_path)

accuracy_values = results_df['metrics/mAP50(B)']
precision_values = results_df['metrics/precision(B)']
recall_values = results_df['metrics/recall(B)']

f1_scores = 2 * (precision_values * recall_values) / (precision_values + recall_values)

average_accuracy = accuracy_values.mean()
average_precision = precision_values.mean()
average_recall = recall_values.mean()
average_f1_score = f1_scores.mean()

metrics = {
    'Model': ['YOLOv8'],
    'Accuracy': [f"{average_accuracy:.2%}"],
    'Precision': [f"{average_precision:.2%}"],
    'Recall': [f"{average_recall:.2%}"],
    'F1 score': [f"{average_f1_score:.2%}"]
}

average_metrics_df = pd.DataFrame(metrics)

print(tabulate(average_metrics_df, headers='keys', tablefmt='grid'))

In [ ]:
import numpy as np

csv_file_path = '/kaggle/working/runs/detect/train/results.csv'
data = np.genfromtxt(csv_file_path, delimiter=',', skip_header=1)
print(data)

In [ ]:


import os
import pandas as pd
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 8))
plt.plot(data[:, 0], data[:, 1], label='Train Box Loss')
plt.plot(data[:, 0], data[:, 8], label='Val Box Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Box Loss')

plt.subplot(2, 2, 2)
plt.plot(data[:, 0], data[:, 2], label='Training Loss')
plt.plot(data[:, 0], data[:, 9], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Training and Validation Loss')


plt.tight_layout()
plt.show()
